<a href="https://colab.research.google.com/github/iguchi-lab/Verification-Platform/blob/main/221120_Verification_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 必要なモジュールのインストール
!pip3 install git+https://github.com/BRI-EES-House/pyhees.git
!pip3 install git+https://github.com/iguchi-lab/jjj_experiment.git@fix01

#以下は、ライブラリをアップグレードする場合に実行する。
#!pip3 install pandas --upgrade
#!pip3 install numpy --upgrade
#!pip3 install scipy --upgrade
#!pip3 install setuptools --upgrade
#!pip3 install jedi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/BRI-EES-House/pyhees.git to /tmp/pip-req-build-39pq0a90
  Running command git clone -q https://github.com/BRI-EES-House/pyhees.git /tmp/pip-req-build-39pq0a90
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 10.4 MB 37.5 MB/s 
     |████████████████████████████████| 503 kB 45.9 MB/s 
     |████████████████████████████████| 26.1 MB 78.0 MB/s 
     |████████████████████████████████| 583 kB 60.5 MB/s 
     |████████████████████████████████| 129 kB 53.0 MB/s 
  Created wheel for pyhees: filename=pyhees-3.3.0-py3-none-any.whl size=211316196 sha256=9c12b3f1ae6f78a4560623a2375166693bc204e03c2e78370b2163079974b923
  Stored in directory: /tmp/pip-ephem-wheel-cache-ptcc0mn0/wheels/78/8b/9b/b70311450ab12dab973c958e9ae04fdf77377fbf97c7c4c2c0
Successfully built pyhees
  Attempting uninstall: six
    Found existing installati

In [ ]:
import numpy as np
import json
import os
import sys

import pprint

from pyhees.section2_1_b import get_f_prim

from pyhees.section4_1 import calc_heating_load, calc_cooling_load, get_virtual_heating_devices, get_alpha_UT_H_A, calc_E_E_H_hs_A_d_t, calc_E_E_C_hs_d_t
from pyhees.section4_1_a import calc_heating_mode

# ダクト式セントラル空調機
import pyhees.section4_2_b as dc_spec

# 床下
import pyhees.section3_1 as ld
from pyhees.section3_2 import calc_r_env, get_Q_dash, get_mu_H, get_mu_C

import jjjexperiment.calc
import jjjexperiment.constants
import jjjexperiment.input

import pandas as pd
from datetime import datetime

#@title 検証用プラットフォーム入力欄

input_data = {}

#@markdown # <font color="orange">**＜① 計算条件名＞**</font>
#@markdown #### <font color="lightgreen">**計算条件名**</font>
case_name = 'test' #@param {type:"string"}      

#@markdown ---
#@markdown # <font color="orange">**＜② 外部ファイル名の入力（ある場合のみ）＞**</font>
#@markdown #### <font color="lightgreen">**気象データファイル名**</font>
climateFile = '-' #@param {type:"string"}
#@markdown #### <font color="lightgreen">**外気条件データファイル名**</font>
outdoorFile = '-' #@param {type:"string"}

#@markdown ---
#@markdown # <font color="orange">**＜③ 計算時定数＞**</font>
#@markdown #### <font color="lightgreen">**最大暖房出力時の熱源機の出口における空気温度の最大値の上限値**</font>
Theta_hs_out_max_H_d_t_limit = 45.0       #@param {type:"number"}
input_data['Theta_hs_out_max_H_d_t_limit'] = Theta_hs_out_max_H_d_t_limit
#@markdown #### <font color="lightgreen">**最大冷房出力時の熱源機の出口における空気温度の最低値の下限値**</font>
Theta_hs_out_min_C_d_t_limit = 15.0       #@param {type:"number"}
input_data['Theta_hs_out_min_C_d_t_limit'] = Theta_hs_out_min_C_d_t_limit
#@markdown #### <font color="lightgreen">**デフロストに関する暖房出力補正係数（ダクトセントラル空調機）**</font>
C_df_H_d_t_defrost_ductcentral = 0.77     #@param {type:"number"}
input_data['C_df_H_d_t_defrost_ductcentral'] = C_df_H_d_t_defrost_ductcentral
#@markdown #### <font color="lightgreen">**デフロスト発生外気温度（ダクトセントラル空調機）**</font>
defrost_temp_ductcentral = 5.0            #@param {type:"number"}
input_data['defrost_temp_ductcentral'] = defrost_temp_ductcentral
#@markdown #### <font color="lightgreen">**デフロスト発生外気相対湿度（ダクトセントラル空調機）**</font>
defrost_humid_ductcentral = 80.0          #@param {type:"number"}
input_data['defrost_humid_ductcentral'] = defrost_humid_ductcentral
#@markdown #### <font color="lightgreen">**ダクトiの線熱損失係数**</font>
phi_i = 0.49                              #@param {type:"number"}
input_data['phi_i'] = phi_i
#@markdown #### <font color="lightgreen">**暖房時の送風機の設計風量に関する係数**</font>
C_V_fan_dsgn_H = 0.79                     #@param {type:"number"}
input_data['C_V_fan_dsgn_H'] = C_V_fan_dsgn_H
#@markdown #### <font color="lightgreen">**冷房時の送風機の設計風量に関する係数**</font>
C_V_fan_dsgn_C = 0.79                     #@param {type:"number"}
input_data['C_V_fan_dsgn_C'] = C_V_fan_dsgn_C
#@markdown #### <font color="lightgreen">**デフロストに関する暖房出力補正係数（ルームエアコンディショナー）**</font>
C_df_H_d_t_defrost_rac = 0.77             #@param {type:"number"}
input_data['C_df_H_d_t_defrost_rac'] = C_df_H_d_t_defrost_rac
#@markdown #### <font color="lightgreen">**デフロスト発生外気温度（ルームエアコンディショナー）**</font>
defrost_temp_rac = 5.0                    #@param {type:"number"}
input_data['defrost_temp_rac'] = defrost_temp_rac
#@markdown #### <font color="lightgreen">**デフロスト発生外気相対湿度（ルームエアコンディショナー）**</font>
defrost_humid_rac = 80.0                  #@param {type:"number"}
input_data['defrost_humid_rac'] = defrost_humid_rac
#@markdown ####<font color="lightgreen">**室内機吸い込み湿度に関する冷房出力補正係数**</font>
C_hm_C = 1.15                             #@param {type:"number"}
input_data['C_hm_C'] = C_hm_C
#@markdown #### <font color="lightgreen">**定格冷房能力の最大値**</font>
q_rtd_C_limit = 5600                      #@param {type:"number"}
input_data['q_rtd_C_limit'] = q_rtd_C_limit

jjjexperiment.constants.set_constants(input_data)

#@markdown ---
#@markdown # <font color="orange">**＜④ 基本情報＞**</font>
#@markdown #### <font color="lightgreen">**面積の合計 [m2]**</font>
A_A = 120.08  #@param {type:"number"}
input_data['A_A'] = A_A
#@markdown #### <font color="lightgreen">**主たる居室の面積 [m2]**</font>
A_MR = 29.81  #@param {type:"number"}
input_data['A_MR'] = A_MR
#@markdown #### <font color="lightgreen">**その他の居室の面積[m2]**</font>
A_OR = 51.34  #@param {type:"number"}
input_data['A_OR'] = A_OR
#@markdown #### <font color="lightgreen">**地域区分**</font>
region = 6    #@param [1, 2, 3, 4, 5, 6] {type:"raw"}
input_data['region'] = region

type, tatekata, A_A, A_MR, A_OR, region, sol_region = jjjexperiment.input.get_basic(input_data)

#@markdown ---
#@markdown # <font color="orange">**＜⑤ 外皮条件＞**</font>
#@markdown #### <font color="lightgreen">**外皮面積 [m2]**</font>
A_env = 307.51    #@param {type:"number"}
input_data['A_env'] = A_env
#@markdown #### <font color="lightgreen">**外皮平均熱貫流率 UA [W/(m2・K)]**</font>
U_A = 0.87        #@param {type:"number"}
input_data['U_A'] = U_A
#@markdown #### <font color="lightgreen">**冷房期平均日射熱取得率ηAC**</font>
eta_A_C = 2.8     #@param {type:"number"}
input_data['eta_A_C'] = eta_A_C
#@markdown #### <font color="lightgreen">**暖房期平均日射熱取得率ηAH**</font>
eta_A_H = 4.3     #@param {type:"number"}
input_data['eta_A_H'] = eta_A_H

#@markdown ---
#@markdown # <font color="orange">**＜⑥ その他＞**</font>
#@markdown #### <font color="lightgreen">**床下空間を経由して外気を導入する換気方式の利用 （☐：評価しない or ☑：評価する）**</font>
underfloor_ventilation = False                    #@param {type:"boolean"}
if underfloor_ventilation:    
    input_data['underfloor_ventilation'] = '2'
else:                                       
    input_data['underfloor_ventilation'] = '1'
#@markdown #### <font color="lightgreen">**外気が経由する床下の面積の割合 [%]**</font>
r_A_ufvnt = 100.0                                 #@param {type:"number"}
input_data['r_A_ufvnt'] = r_A_ufvnt
#@markdown #### <font color="lightgreen">**床下空間の断熱 （☐：断熱区画外 or ☑：断熱区画内）**</font>
underfloor_insulation = False                     #@param {type:"boolean"}
if underfloor_insulation:     
    input_data['underfloor_insulation'] = '2'
else:                                       
    input_data['underfloor_insulation'] = '1'
#@markdown #### <font color="lightgreen">**空調空気を床下を通して給気する （☐：床下を通して給気しない or ☑：床下を通して給気する）**</font>
underfloor_air_conditioning_air_supply = False    #@param {type:"boolean"}
if underfloor_air_conditioning_air_supply:     
    input_data['underfloor_air_conditioning_air_supply'] = '2'
else:                                       
    input_data['underfloor_air_conditioning_air_supply'] = '1'

#@markdown #### <font color="lightgreen">**全体風量を固定する （☐：固定しない or ☑：固定する）**</font>
hs_CAV = False                                    #@param {type:"boolean"}
if hs_CAV:     
    input_data['hs_CAV'] = '2'
else:                                       
    input_data['hs_CAV'] = '1'
ENV, NV_MR, NV_OR, TS, r_A_ufvnt, underfloor_insulation, underfloor_air_conditioning_air_supply, hs_CAV = jjjexperiment.input.get_env(input_data)

#@markdown ---
#@markdown # <font color="orange">**＜⑦ 暖房全般＞**</font>
#@markdown #### <font color="lightgreen">**暖房設備の種類 （ダクト式セントラル空調機 or ルームエアコンディショナ活用型全館空調）**</font>
input_data['H_A']  = {}
H_A_type = "\u30C0\u30AF\u30C8\u5F0F\u30BB\u30F3\u30C8\u30E9\u30EB\u7A7A\u8ABF\u6A5F" #@param ["ダクト式セントラル空調機", "ルームエアコンディショナ活用型全館空調"] {type:"string"}
if H_A_type == "ダクト式セントラル空調機":
    input_data['H_A']['type'] = 1
else:
    input_data['H_A']['type'] = 2
#@markdown #### <font color="lightgreen">**ダクトが通過する空間　（全てもしくは一部が断熱区画外である or 全て断熱区画内である）**</font>
H_A_duct_insulation = "全てもしくは一部が断熱区画外である"    #@param ["全てもしくは一部が断熱区画外である", "全て断熱区画内である"] {type:"string"}
if H_A_duct_insulation == "全てもしくは一部が断熱区画外である":
    input_data['H_A']['duct_insulation'] = 1
else:
    input_data['H_A']['duct_insulation'] = 2
#@markdown #### <font color="lightgreen">**VAV方式　（採用しない or 採用する）**</font>
H_A_VAV = "採用しない" #@param ["採用しない", "採用する"] {type:"string"}
if H_A_VAV == "採用しない":
    input_data['H_A']['VAV'] = 1
else:
    input_data['H_A']['VAV'] = 2
#@markdown #### <font color="lightgreen">**全般換気機能　（あり or なし）**</font>
H_A_general_ventilation = "\u3042\u308A" #@param ["あり", "なし"] {type:"string"}
if H_A_general_ventilation == "あり":
    input_data['H_A']['general_ventilation'] = 1
else:
    input_data['H_A']['general_ventilation'] = 2
#@markdown #### <font color="orange">**―設計風量―**</font>
#@markdown #### <font color="lightgreen">**設計風量（入力しない or 入力する）**</font>
H_A_input_V_hs_dsgn_H = "入力しない" #@param ["入力しない", "入力する"] {type:"string"}
if H_A_input_V_hs_dsgn_H == "入力しない":
    input_data['H_A']['input_V_hs_dsgn_H'] = 1
else:
    input_data['H_A']['input_V_hs_dsgn_H'] = 2
#@markdown #### <font color="lightblue">**設計風量 [m3/h]（入力する場合のみ）**</font>
H_A_V_hs_dsgn_H = 500.0    #@param {type:"number"}
input_data['H_A']['input_V_hs_dsgn_H'] = H_A_V_hs_dsgn_H
#@markdown #### <font color="orange">**―ファンの消費電力―**</font>
#@markdown #### <font color="lightgreen">**ファンの比消費電力（入力しない or 入力する）**</font>
H_A_input_f_SFP_H = "入力しない" #@param ["入力しない", "入力する"] {type:"string"}
if H_A_input_f_SFP_H == "入力しない":
    input_data['H_A']['input_f_SFP_H'] = 1
else:
    input_data['H_A']['input_f_SFP_H'] = 2
#@markdown #### <font color="lightblue">**ファンの比消費電力W [m3/h]（入力する場合のみ）**</font>
H_A_input_f_SFP_H = 0.144    #@param {type:"number"}
input_data['H_A']['f_SFP_H'] = H_A_input_f_SFP_H

#@markdown ---
#@markdown # <font color="orange">**＜⑦-1 暖房（ダクト式セントラル空調機の場合のみ）＞**</font>
#@markdown #### <font color="orange">**―機器仕様の入力―**</font>
#@markdown #### <font color="lightgreen">**機器仕様の入力（入力しない or 定格能力試験の値を入力する or 定格能力試験と中間能力試験の値を入力する）**</font>
H_A_input = "入力しない" #@param ["入力しない", "定格能力試験の値を入力する", "定格能力試験と中間能力試験の値を入力する"] {type:"string"}
if H_A_input == "入力しない":
    input_data['H_A']['input'] = 1
elif H_A_input == "定格能力試験の値を入力する":
    input_data['H_A']['input'] = 2
else:
    input_data['H_A']['input'] = 3
#@markdown #### <font color="lightblue">**定格暖房能力試験 能力 [W]（入力する場合のみ）**</font>
H_A_q_hs_rtd_H = 0.0    #@param {type:"number"}
input_data['H_A']['q_hs_rtd_H'] = H_A_q_hs_rtd_H
#@markdown #### <font color="lightblue">**定格暖房能力試験 消費電力 [W]（入力する場合のみ）**</font>
H_A_P_hs_rtd_H = 0.0    #@param {type:"number"}
input_data['H_A']['P_hs_rtd_H'] = H_A_P_hs_rtd_H
#@markdown #### <font color="lightblue">**定格暖房能力試験 風量 [m3/h]（入力する場合のみ）**</font>
H_A_V_fan_rtd_H = 0.0   #@param {type:"number"}
input_data['H_A']['V_fan_rtd_H'] = H_A_V_fan_rtd_H
#@markdown #### <font color="lightblue">**定格暖房能力試験 室内側送風機の消費電力 [W]（入力する場合のみ）**</font>
H_A_P_fan_rtd_H = 0.0   #@param {type:"number"}
input_data['H_A']['P_fan_rtd_H'] = H_A_P_fan_rtd_H
#@markdown #### <font color="lightblue">**中間暖房能力試験 能力 [W]（入力する場合のみ）**</font>
H_A_q_hs_mid_H = 0.0    #@param {type:"number"}
input_data['H_A']['q_hs_mid_H'] = H_A_q_hs_mid_H
#@markdown #### <font color="lightblue">**中間暖房能力試験 消費電力 [W]（入力する場合のみ）**</font>
H_A_P_hs_mid_H = 0.0    #@param {type:"number"}
input_data['H_A']['P_hs_mid_H'] = H_A_P_hs_mid_H
#@markdown #### <font color="lightblue">**中間暖房能力試験 風量 [m3/h]（入力する場合のみ）**</font>
H_A_V_fan_mid_H = 0.0   #@param {type:"number"}
input_data['H_A']['V_fan_mid_H'] = H_A_V_fan_mid_H
#@markdown #### <font color="lightblue">**中間暖房能力試験 室内側送風機の消費電力 [W]（入力する場合のみ）**</font>
H_A_P_fan_mid_H = 0.0   #@param {type:"number"}
input_data['H_A']['P_fan_mid_H'] = H_A_P_fan_mid_H

mode_H, H_A, H_MR, H_OR, H_HS = jjjexperiment.input.get_heating(input_data, region, A_A)

#@markdown ---
#@markdown # <font color="orange">**＜⑦-2 暖房（ルームエアコンディショナ活用型全館空調の場合のみ）＞**</font>
#@markdown #### <font color="orange">**―暖房能力の入力―**</font>
#@markdown #### <font color="lightgreen">**暖房能力の入力（面積から能力を算出 or 性能を直接入力）**</font>
H_A_input_rac_performance = "面積から能力を算出" #@param ["面積から能力を算出", "性能を直接入力"] {type:"string"}
if H_A_input_rac_performance == "面積から能力を算出":
    input_data['H_A']['input_rac_performance'] = 1
else:
    input_data['H_A']['input_rac_performance'] = 2
#@markdown #### <font color="lightblue">**暖房定格能力 [W]（入力する場合のみ）**</font>
H_A_q_rac_rtd_H = 0.0   #@param {type:"number"}
input_data['H_A']['q_rac_rtd_H'] = H_A_q_rac_rtd_H
#@markdown #### <font color="lightblue">**暖房最大能力 [W]（入力する場合のみ）**</font>
H_A_q_rac_max_H = 0.0   #@param {type:"number"}
input_data['H_A']['q_rac_max_H'] = H_A_q_rac_max_H
#@markdown #### <font color="lightblue">**定格エネルギー効率 [-]（入力する場合のみ）**</font>
e_rtd_H = 0.0   #@param {type:"number"}
input_data['H_A']['e_rac_rtd_H'] = e_rtd_H
#@markdown #### <font color="lightgreen">**小能力時高効率型コンプレッサー（評価しない or 搭載する）**</font>
H_A_dualcompressor = "評価しない" #@param ["評価しない", "搭載する"] {type:"string"}
if H_A_dualcompressor == "評価しない":
    input_data['H_A']['dualcompressor'] = 1
else:
    input_data['H_A']['dualcompressor'] = 2
#@markdown #### <font color="orange">**―設置方法の入力―**</font>
#@markdown #### <font color="lightgreen">**設置方法の入力（設置方法を入力する or 補正係数を直接入力する）**</font>
H_A_input_C_af_H = "設置方法を入力する" #@param ["設置方法を入力する", "補正係数を直接入力する"] {type:"string"}
if H_A_input_C_af_H == "設置方法を入力する":
    input_data['H_A']['input_C_af_H'] = 1
else:
    input_data['H_A']['input_C_af_H'] = 2
#@markdown #### <font color="lightblue">**専用チャンバーに格納される方式（該当しない or 該当する）**</font>
H_A_dedicated_chamber = "該当しない" #@param ["該当しない", "該当する"] {type:"string"}
if H_A_dedicated_chamber == "該当しない":
    input_data['H_A']['dedicated_chamber'] = 1
else:
    input_data['H_A']['dedicated_chamber'] = 2
#@markdown #### <font color="lightblue">**フィン向きが中央位置に固定される方式（該当しない or 該当する）**</font>
H_A_fixed_fin_direction = "該当しない" #@param ["該当しない", "該当する"] {type:"string"}
if H_A_fixed_fin_direction == "該当しない":
    input_data['H_A']['fixed_fin_direction'] = 1
else:
    input_data['H_A']['fixed_fin_direction'] = 2
#@markdown #### <font color="lightblue">**室内機吹き出し風量に関する暖房出力補正係数の入力（入力する場合のみ）**</font>
H_A_C_af_H = 0.0   #@param {type:"number"}
input_data['H_A']['C_af_H'] = H_A_C_af_H

#@markdown ---
#@markdown # <font color="orange">**＜⑧ 冷房全般＞**</font>
#@markdown #### <font color="lightgreen">**冷房設備の種類 （ダクト式セントラル空調機 or ルームエアコンディショナ活用型全館空調）**</font>
input_data['C_A']  = {}
C_A_type = "\u30C0\u30AF\u30C8\u5F0F\u30BB\u30F3\u30C8\u30E9\u30EB\u7A7A\u8ABF\u6A5F" #@param ["ダクト式セントラル空調機", "ルームエアコンディショナ活用型全館空調"] {type:"string"}
if C_A_type == "ダクト式セントラル空調機":
    input_data['C_A']['type'] = 1
else:
    input_data['C_A']['type'] = 2
#@markdown #### <font color="lightgreen">**ダクトが通過する空間　（全てもしくは一部が断熱区画外である or 全て断熱区画内である）**</font>
C_A_duct_insulation = "全てもしくは一部が断熱区画外である"    #@param ["全てもしくは一部が断熱区画外である", "全て断熱区画内である"] {type:"string"}
if C_A_duct_insulation == "全てもしくは一部が断熱区画外である":
    input_data['C_A']['duct_insulation'] = 1
else:
    input_data['C_A']['duct_insulation'] = 2
#@markdown #### <font color="lightgreen">**VAV方式　（採用しない or 採用する）**</font>
C_A_VAV = "採用しない" #@param ["採用しない", "採用する"] {type:"string"}
if C_A_VAV == "採用しない":
    input_data['C_A']['VAV'] = 1
else:
    input_data['C_A']['VAV'] = 2
#@markdown #### <font color="lightgreen">**全般換気機能　（あり or なし）**</font>
C_A_general_ventilation = "あり" #@param ["あり", "なし"] {type:"string"}
if C_A_general_ventilation == "あり":
    input_data['C_A']['general_ventilation'] = 1
else:
    input_data['C_A']['general_ventilation'] = 2
#@markdown #### <font color="orange">**―設計風量―**</font>
#@markdown #### <font color="lightgreen">**設計風量（入力しない or 入力する）**</font>
C_A_input_V_hs_dsgn_C = "入力しない" #@param ["入力しない", "入力する"] {type:"string"}
if C_A_input_V_hs_dsgn_C == "入力しない":
    input_data['C_A']['input_V_hs_dsgn_C'] = 1
else:
    input_data['C_A']['input_V_hs_dsgn_C'] = 2
#@markdown #### <font color="lightblue">**設計風量 [m3/h]（入力する場合のみ）**</font>
C_A_V_hs_dsgn_C = 500.0    #@param {type:"number"}
input_data['C_A']['V_hs_dsgn_C'] = C_A_V_hs_dsgn_C
#@markdown #### <font color="orange">**―ファンの消費電力―**</font>
#@markdown #### <font color="lightgreen">**ファンの比消費電力（入力しない or 入力する）**</font>
C_A_input_f_SFP_C = "入力しない" #@param ["入力しない", "入力する"] {type:"string"}
if C_A_input_f_SFP_C == "入力しない":
    input_data['C_A']['input_f_SFP_C'] = 1
else:
    input_data['C_A']['input_f_SFP_C'] = 2
#@markdown #### <font color="lightblue">**ファンの比消費電力W [m3/h]（入力する場合のみ）**</font>
C_A_f_SFP_C = 0.144    #@param {type:"number"}
input_data['C_A']['f_SFP_C'] = C_A_f_SFP_C

#@markdown ---
#@markdown # <font color="orange">**＜⑧-1 冷房（ダクト式セントラル空調機の場合のみ）＞**</font>
#@markdown #### <font color="orange">**―機器仕様の入力―**</font>
#@markdown #### <font color="lightgreen">**機器仕様の入力（入力しない or 定格能力試験の値を入力する or 定格能力試験と中間能力試験の値を入力する）**</font>
C_A_input = "入力しない" #@param ["入力しない", "定格能力試験の値を入力する", "定格能力試験と中間能力試験の値を入力する"] {type:"string"}
if C_A_input == "入力しない":
    input_data['C_A']['input'] = 1
elif C_A_input == "定格能力試験の値を入力する":
    input_data['C_A']['input'] = 2
else:
    input_data['C_A']['input'] = 3
#@markdown #### <font color="lightblue">**定格冷房能力試験 能力 [W]（入力する場合のみ）**</font>
C_A_q_hs_rtd_C = 0.0    #@param {type:"number"}
input_data['C_A']['q_hs_rtd_C'] = C_A_q_hs_rtd_C
#@markdown #### <font color="lightblue">**定格冷房能力試験 消費電力 [W]（入力する場合のみ）**</font>
C_A_P_hs_rtd_C = 0.0    #@param {type:"number"}
input_data['C_A']['P_hs_rtd_C'] = C_A_P_hs_rtd_C
#@markdown #### <font color="lightblue">**定格冷房能力試験 風量 [m3/h]（入力する場合のみ）**</font>
C_A_V_fan_rtd_C = 0.0   #@param {type:"number"}
input_data['C_A']['V_fan_rtd_C'] = C_A_V_fan_rtd_C
#@markdown #### <font color="lightblue">**定格冷房能力試験 室内側送風機の消費電力 [W]（入力する場合のみ）**</font>
C_A_P_fan_rtd_C= 0.0   #@param {type:"number"}
input_data['C_A']['P_fan_rtd_C'] = C_A_P_fan_rtd_C
#@markdown #### <font color="lightblue">**中間冷房能力試験 能力 [W]（入力する場合のみ）**</font>
C_A_q_hs_mid_C= 0.0    #@param {type:"number"}
input_data['C_A']['q_hs_mid_C'] = C_A_q_hs_mid_C
#@markdown #### <font color="lightblue">**中間冷房能力試験 消費電力 [W]（入力する場合のみ）**</font>
C_A_P_hs_mid_C = 0.0    #@param {type:"number"}
input_data['C_A']['P_hs_mid_C'] = C_A_P_hs_mid_C
#@markdown #### <font color="lightblue">**中間冷房能力試験 風量 [m3/h]（入力する場合のみ）**</font>
C_A_V_fan_mid_C = 0.0   #@param {type:"number"}
input_data['C_A']['V_fan_mid_C'] = C_A_V_fan_mid_C
#@markdown #### <font color="lightblue">**中間冷房能力試験 室内側送風機の消費電力 [W]（入力する場合のみ）**</font>
C_A_P_fan_mid_C = 0.0   #@param {type:"number"}
input_data['C_A']['P_fan_mid_C'] = C_A_P_fan_mid_C

mode_C, C_A, C_MR, C_OR = jjjexperiment.input.get_cooling(input_data, region, A_A)

#@markdown ---
#@markdown # <font color="orange">**＜⑧-2 冷房（ルームエアコンディショナ活用型全館空調の場合のみ）＞**</font>
#@markdown #### <font color="orange">**―エネルギー消費効率の入力―**</font>
#@markdown #### <font color="lightgreen">**エネルギー消費効率の入力（入力しない or 入力する）**</font>
C_A_input_mode = "入力しない" #@param ["入力しない", "入力する"] {type:"string"}
if C_A_input_mode == "入力しない":
    input_data['C_A']['input_mode'] = 1
else:
    input_data['C_A']['input_mode'] = 2
#@markdown #### <font color="lightblue">**エネルギー消費効率（い or ろ or は）（入力する場合のみ）**</font>
C_A_mode = "は" #@param ["い", "ろ", "は"] {type:"string"}
input_data['C_A']['mode'] = C_A_mode
#@markdown #### <font color="orange">**―冷房能力の入力―**</font>
#@markdown #### <font color="lightgreen">**冷房能力の入力（面積から能力を算出 or 性能を直接入力）**</font>
C_A_input_rac_performance = "面積から能力を算出" #@param ["面積から能力を算出", "性能を直接入力"] {type:"string"}
if C_A_input_rac_performance == "面積から能力を算出":
    input_data['C_A']['input_rac_performance'] = 1
else:
    input_data['C_A']['input_rac_performance'] = 2
#@markdown #### <font color="lightblue">**冷房定格能力 [W]（入力する場合のみ）**</font>
C_A_q_rac_rtd_C = 0.0   #@param {type:"number"}
input_data['C_A']['q_rac_rtd_C'] = C_A_q_rac_rtd_C
#@markdown #### <font color="lightblue">**冷房最大能力 [W]（入力する場合のみ）**</font>
C_A_q_rac_max_C = 0.0   #@param {type:"number"}
input_data['C_A']['q_rac_max_C'] = C_A_q_rac_max_C
#@markdown #### <font color="lightblue">**定格エネルギー効率 [-]（入力する場合のみ）**</font>
e_rtd_C = 0.0   #@param {type:"number"}
input_data['C_A']['e_rac_rtd_C'] = e_rtd_C
#@markdown #### <font color="lightgreen">**小能力時高効率型コンプレッサー（評価しない or 搭載する）**</font>
C_A_dualcompressor = "評価しない" #@param ["評価しない", "搭載する"] {type:"string"}
if C_A_dualcompressor == "評価しない":
    input_data['C_A']['dualcompressor'] = 1
else:
    input_data['C_A']['dualcompressor'] = 2
#@markdown #### <font color="orange">**―設置方法の入力―**</font>
#@markdown #### <font color="lightgreen">**設置方法の入力（設置方法を入力する or 補正係数を直接入力する）**</font>
C_A_input_C_af_C = "設置方法を入力する" #@param ["設置方法を入力する", "補正係数を直接入力する"] {type:"string"}
if C_A_input_C_af_C == "設置方法を入力する":
    input_data['C_A']['input_C_af_C'] = 1
else:
    input_data['C_A']['input_C_af_C'] = 2
#@markdown #### <font color="lightblue">**専用チャンバーに格納される方式（該当しない or 該当する）**</font>
C_A_dedicated_chamber = "該当しない" #@param ["該当しない", "該当する"] {type:"string"}
if C_A_dedicated_chamber == "該当しない":
    input_data['C_A']['dedicated_chamber'] = 1
else:
    input_data['C_A']['dedicated_chamber'] = 2
#@markdown #### <font color="lightblue">**フィン向きが中央位置に固定される方式（該当しない or 該当する）**</font>
C_A_fixed_fin_direction = "該当しない" #@param ["該当しない", "該当する"] {type:"string"}
if C_A_fixed_fin_direction == "該当しない":
    input_data['C_A']['fixed_fin_direction'] = 1
else:
    input_data['C_A']['fixed_fin_direction'] = 2
#@markdown #### <font color="lightblue">**室内機吹き出し風量に関する冷房出力補正係数の入力（入力する場合のみ）**</font>
C_A_C_af_C = 0.0   #@param {type:"number"}
input_data['C_A']['C_af_C'] = C_A_C_af_C

# セントラルルームエアコンディショナの性能の取得
q_rtd_C, q_rtd_H, q_max_C, q_max_H, e_rtd_C, e_rtd_H, dualcompressor_C, dualcompressor_H,\
    input_C_af_C, input_C_af_H = jjjexperiment.input.get_CRAC_spec(input_data)

#@markdown ---
#@markdown # <font color="orange">**＜⑨ 熱交換型換気設備）＞**</font>
#@markdown #### <font color="lightblue">**熱交換型換気設備（設置しない or 設置する）**</font>
input_data['HEX'] = {}

HEX_install = "\u8A2D\u7F6E\u3059\u308B" #@param ["設置しない", "設置する"] {type:"string"}
if HEX_install == "設置しない":
    input_data['HEX']['install'] = 1
else:
    input_data['HEX']['install'] = 2
#@markdown #### <font color="lightblue">**温度交換効率（設置する場合のみ）**</font>
etr_t = 40.0   #@param {type:"number"}
input_data['HEX']['etr_t'] = etr_t

# 熱交換型換気の取得
HEX = jjjexperiment.input.get_heatexchangeventilation(input_data)

# 太陽熱利用の取得
SHC = jjjexperiment.input.get_solarheat()

 # 床面積の合計に対する外皮の部位の面積の合計の比
r_env = calc_r_env(
    method='当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法',
    A_env=ENV['A_env'],
    A_A=A_A
 )

# 熱損失係数（換気による熱損失を含まない）
Q_dash = get_Q_dash(ENV['U_A'], r_env)
# 熱損失係数
Q = ld.get_Q(Q_dash)

# 日射取得係数の取得
mu_H = get_mu_H(ENV['eta_A_H'], r_env)
mu_C = get_mu_C(ENV['eta_A_C'], r_env)

# 実質的な暖房機器の仕様を取得
spec_MR, spec_OR = get_virtual_heating_devices(region, H_MR, H_OR)

# 暖房方式及び運転方法の区分
mode_MR, mode_OR = calc_heating_mode(region=region, H_MR=spec_MR, H_OR=spec_OR)

# 空調空気を床下を通して給気する場合（YUCACO）の「床下空間全体の面積に対する空気を供給する床下空間の面積の比 (-)」
YUCACO_r_A_ufvnt = (8.28+16.56+21.53) / (9.52+1.24+3.31+3.31+1.66+8.28+16.56+21.53)

with open(case_name + '_input.json', 'w') as f:
    json.dump(input_data, f, indent=4)

##### 暖房負荷の取得（MJ/h）
L_H_d_t_i: np.ndarray
"""暖房負荷 [MJ/h]"""

L_H_d_t_i, _ = calc_heating_load(region, sol_region, A_A, A_MR, A_OR, Q, mu_H, mu_C, NV_MR, NV_OR, TS, r_A_ufvnt,
                                     HEX, underfloor_insulation, mode_H, mode_C, spec_MR, spec_OR, mode_MR, mode_OR, SHC)
L_H_d_t: np.ndarray = np.sum(L_H_d_t_i, axis=0)
"""暖房負荷の全区画合計 [MJ/h]"""

##### 冷房負荷の取得（MJ/h）
L_CS_d_t_i: np.ndarray
"""冷房顕熱負荷 [MJ/h]"""
L_CL_d_t_i: np.ndarray
"""冷房潜熱負荷 [MJ/h]"""
L_CS_d_t_i, L_CL_d_t_i = calc_cooling_load(region, A_A, A_MR, A_OR, Q, mu_H, mu_C, NV_MR, NV_OR, r_A_ufvnt,
                                               underfloor_insulation, mode_C, mode_H, mode_MR, mode_OR, TS, HEX)
L_CS_d_t: np.ndarray = np.sum(L_CS_d_t_i, axis=0)
"""冷房顕熱負荷の全区画合計 [MJ/h]"""
L_CL_d_t: np.ndarray = np.sum(L_CL_d_t_i, axis=0)
"""冷房潜熱負荷の全区画合計 [MJ/h]"""

##### 暖房消費電力の計算（kWh/h）
V_rac_fan_rtd_H: float = dc_spec.get_V_fan_rtd_H(q_rtd_H)
"""定格暖房能力運転時の送風機の風量(m3/h)"""

if H_A['type'] == 'ダクト式セントラル空調機':
    if 'V_hs_dsgn_H' in H_A:
        V_hs_dsgn_H = H_A['V_hs_dsgn_H']
    else:
        V_hs_dsgn_H = dc_spec.get_V_fan_dsgn_C(H_A['V_fan_rtd_H'])
elif H_A['type'] == 'ルームエアコンディショナ活用型全館空調システム':
        V_hs_dsgn_H = dc_spec.get_V_fan_dsgn_C(V_rac_fan_rtd_H)
else: 
    raise Exception("暖房方式が不正です。")
"""暖房時の送風機の設計風量(m3/h)"""

P_rac_fan_rtd_H: float = dc_spec.get_P_fan_rtd_H(V_rac_fan_rtd_H)
"""定格暖房能力運転時の送風機の消費電力(W)"""

V_hs_dsgn_C: float = None
"""冷房時の送風機の設計風量(m3/h)"""

Q_UT_H_d_t_i: np.ndarray
"""暖房設備機器等の未処理暖房負荷(MJ/h)"""

E_E_H_d_t: np.ndarray
"""日付dの時刻tにおける1時間当たりの暖房時の消費電力量(kWh/h)"""

_, Q_UT_H_d_t_i, _, _, Theta_hs_out_d_t, Theta_hs_in_d_t, Theta_ex_d_t, _, _, V_hs_supply_d_t, V_hs_vent_d_t, C_df_H_d_t =\
    jjjexperiment.calc.calc_Q_UT_A(A_A, A_MR, A_OR, ENV['A_env'], mu_H, mu_C,
        H_A['q_hs_rtd_H'], None,
        q_rtd_H, q_rtd_C, q_max_H, q_max_C, V_hs_dsgn_H, V_hs_dsgn_C, Q, H_A['VAV'], H_A['general_ventilation'], hs_CAV,
        H_A['duct_insulation'], region, L_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i,
        H_A['type'], input_C_af_H, input_C_af_C,
        underfloor_insulation, underfloor_air_conditioning_air_supply, YUCACO_r_A_ufvnt, climateFile, outdoorFile)

E_E_H_d_t: np.ndarray
"""日付dの時刻tにおける1時間当たりの暖房時の消費電力量(kWh/h)"""

E_E_H_d_t, q_hs_H_d_t, E_E_fan_H_d_t = jjjexperiment.calc.calc_E_E_H_d_t(
    Theta_hs_out_d_t = Theta_hs_out_d_t,
    Theta_hs_in_d_t = Theta_hs_in_d_t,
    Theta_ex_d_t = Theta_ex_d_t,
    V_hs_supply_d_t = V_hs_supply_d_t,
    V_hs_vent_d_t = V_hs_vent_d_t,
    C_df_H_d_t = C_df_H_d_t,
    V_hs_dsgn_H = V_hs_dsgn_H,
    EquipmentSpec = H_A['EquipmentSpec'],
    q_hs_rtd_H = H_A['q_hs_rtd_H'],
    P_hs_rtd_H = H_A['P_hs_rtd_H'],
    V_fan_rtd_H = H_A['V_fan_rtd_H'],
    P_fan_rtd_H = H_A['P_fan_rtd_H'],
    q_hs_mid_H = H_A['q_hs_mid_H'],
    P_hs_mid_H = H_A['P_hs_mid_H'],
    V_fan_mid_H = H_A['V_fan_mid_H'],
    P_fan_mid_H = H_A['P_fan_mid_H'],
    q_hs_min_H = H_A['q_hs_min_H'],
    region = region,
    type = H_A['type'],
    q_rtd_C = q_rtd_C,
    q_rtd_H = q_rtd_H,
    P_rac_fan_rtd_H = P_rac_fan_rtd_H,
    q_max_C = q_max_C,
    q_max_H = q_max_H,
    e_rtd_H = e_rtd_H,
    dualcompressor_H = dualcompressor_H,
    input_C_af_H = input_C_af_H,
    f_SFP_H = H_A['f_SFP_H'])

alpha_UT_H_A: float = get_alpha_UT_H_A(region)
"""未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数"""
Q_UT_H_A_d_t: np.ndarray = np.sum(Q_UT_H_d_t_i, axis=0)
"""未処理暖房負荷の全機器合計(MJ/h)"""
E_UT_H_d_t: np.ndarray = Q_UT_H_A_d_t * alpha_UT_H_A
"""未処理暖房負荷の設計一次エネルギー消費量相当値(MJ/h)"""

##### 冷房消費電力の計算（kWh/h）

V_fan_rtd_C: float = dc_spec.get_V_fan_rtd_C(q_rtd_C)
"""定格冷房能力運転時の送風機の風量(m3/h)"""

if C_A['type'] == 'ダクト式セントラル空調機':
    if 'V_hs_dsgn_C' in C_A:
        V_hs_dsgn_C = C_A['V_hs_dsgn_C']
    else:
        V_hs_dsgn_C = dc_spec.get_V_fan_dsgn_C(C_A['V_fan_rtd_C'])
elif C_A['type'] == 'ルームエアコンディショナ活用型全館空調システム':
    V_hs_dsgn_C = dc_spec.get_V_fan_dsgn_C(V_fan_rtd_C)
else: 
    raise Exception("冷房方式が不正です。")
"""冷房時の送風機の設計風量(m3/h)"""

P_rac_fan_rtd_C: float = dc_spec.get_P_fan_rtd_C(V_fan_rtd_C)
"""定格冷房能力運転時の送風機の消費電力(W)"""

V_hs_dsgn_H: float = None
"""暖房時の送風機の設計風量(m3/h)"""

E_C_UT_d_t: np.ndarray
"""冷房設備の未処理冷房負荷の設計一次エネルギー消費量相当値(MJ/h)"""

E_C_UT_d_t: np.ndarray
"""日付dの時刻tにおける1時間当たりの冷房時の消費電力量(kWh/h)"""

E_C_UT_d_t, _, _, _, Theta_hs_out_d_t, Theta_hs_in_d_t, Theta_ex_d_t, X_hs_out_d_t, X_hs_in_d_t, V_hs_supply_d_t, V_hs_vent_d_t, _\
    = jjjexperiment.calc.calc_Q_UT_A(A_A, A_MR, A_OR, ENV['A_env'], mu_H, mu_C,
        None, C_A['q_hs_rtd_C'],
        q_rtd_H, q_rtd_C, q_max_H, q_max_C, V_hs_dsgn_H, V_hs_dsgn_C, Q, C_A['VAV'], C_A['general_ventilation'], hs_CAV,
        C_A['duct_insulation'], region, L_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i,
        C_A['type'], input_C_af_H, input_C_af_C,
        underfloor_insulation, underfloor_air_conditioning_air_supply, YUCACO_r_A_ufvnt, climateFile, outdoorFile)

E_E_C_d_t, E_E_fan_C_d_t, q_hs_CS_d_t, q_hs_CL_d_t = jjjexperiment.calc.get_E_E_C_d_t(
    Theta_hs_out_d_t = Theta_hs_out_d_t,
    Theta_hs_in_d_t = Theta_hs_in_d_t,
    Theta_ex_d_t = Theta_ex_d_t,
    X_hs_out_d_t = X_hs_out_d_t,
    X_hs_in_d_t = X_hs_in_d_t,
    V_hs_supply_d_t = V_hs_supply_d_t,
    V_hs_vent_d_t = V_hs_vent_d_t,
    V_hs_dsgn_C = V_hs_dsgn_C,
    EquipmentSpec = C_A['EquipmentSpec'],
    q_hs_rtd_C = C_A['q_hs_rtd_C'],
    P_hs_rtd_C = C_A['P_hs_rtd_C'],
    V_fan_rtd_C = C_A['V_fan_rtd_C'],
    P_fan_rtd_C = C_A['P_fan_rtd_C'],
    q_hs_mid_C = C_A['q_hs_mid_C'],
    P_hs_mid_C = C_A['P_hs_mid_C'],
    V_fan_mid_C = C_A['V_fan_mid_C'],
    P_fan_mid_C = C_A['P_fan_mid_C'],
    q_hs_min_C = C_A['q_hs_min_C'],
    region = region,
    type = C_A['type'],
    q_rtd_C = q_rtd_C,
    e_rtd_C = e_rtd_C,
    P_rac_fan_rtd_C = P_rac_fan_rtd_C,
    q_max_C = q_max_C,
    dualcompressor_C = dualcompressor_C,
    input_C_af_C = input_C_af_C,
    f_SFP_C = C_A['f_SFP_C'])

##### 計算結果のまとめ
pprint.pprint(input_data)

f_prim: float       = get_f_prim()                              #電気の量 1kWh を熱量に換算する係数(kJ/kWh)
E_H_d_t: np.ndarray = E_E_H_d_t * f_prim / 1000 + E_UT_H_d_t    #1 時間当たりの暖房設備の設計一次エネルギー消費量(MJ/h)
E_C_d_t: np.ndarray = E_E_C_d_t * f_prim / 1000 + E_C_UT_d_t    #1 時間当たりの冷房設備の設計一次エネルギー消費量(MJ/h)
E_H                 = np.sum(E_H_d_t)                           #1 年当たりの暖房設備の設計一次エネルギー消費量(MJ/年)
E_C                 = np.sum(E_C_d_t)                           #1 年当たりの冷房設備の設計一次エネルギー消費量(MJ/年)

df_output1 = pd.DataFrame(index = ['合計値'])
df_output1['E_H [MJ/year]'] = E_H
df_output1['E_C [MJ/year]'] = E_C
df_output1.to_csv(case_name + '_output1.csv', encoding = 'cp932')
print('E_H [MJ/year]:', E_H, ', E_C [MJ/year]:', E_C)

df_output2 = pd.DataFrame(index = pd.date_range(datetime(2022, 1, 1, 1, 0, 0), datetime(2023, 1, 1, 0, 0, 0), freq = 'h'))
df_output2['E_H_d_t [MJ/h]']        = E_H_d_t
df_output2['E_C_d_t [MJ/h]']        = E_C_d_t
df_output2['E_E_H_d_t [kWh/h]']     = E_E_H_d_t
df_output2['E_E_C_d_t [kWh/h]']     = E_E_C_d_t
df_output2['E_UT_H_d_t [MJ/h]']     = E_UT_H_d_t
df_output2['E_UT_C_d_t [MJ/h]']     = E_C_UT_d_t
df_output2['L_H_d_t [MJ/h]']        = L_H_d_t
df_output2['L_CS_d_t [MJ/h]']       = L_CS_d_t
df_output2['L_CL_d_t [MJ/h]']       = L_CL_d_t
df_output2['E_E_fan_H_d_t [kWh/h]'] = E_E_fan_H_d_t
df_output2['q_hs_H_d_t [Wh/h]']     = q_hs_H_d_t
df_output2['E_E_fan_C_d_t [kWh/h]'] = E_E_fan_C_d_t
df_output2['q_hs_CS_d_t [Wh/h]']    = q_hs_CS_d_t
df_output2['q_hs_CL_d_t [Wh/h]']    = q_hs_CL_d_t
df_output2.to_csv(case_name + '_output2.csv', encoding = 'cp932')
df_output2

In [1]:
from google.colab import files
files.upload()

{}